# I’m Something of a Painter Myself
I wanted to be a picasso like painter, but haven't got enought talent to paint. So using the GAN I am noe transforming the photo images in this competition into a monet. I am using the tfrecords to get the better out of the model in training. I tried to explain every step to the best of my knowledge! Please do support if found interesting 

Also I took some references from the following resources 

[Link 1](https://www.kaggle.com/ryanholbrook/tfrecords-basics)
[Link 2](https://www.kaggle.com/drzhuzhe/monet-cyclegan-tutorial)
[Link 3](https://kozodoi.me/python/deep%20learning/computer%20vision/competitions/2020/08/30/pre-training.html)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os
import re
import cv2
import math
import random

In [ ]:
import keras
from keras import *
from keras.layers import *
import tensorflow_addons as tfa

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()

Data is stored as tfrecords for faster training and processing. `tf.io.gfile.glob` returns the list of files that manages the pattern given. For eg: "*.tfrec"

In [ ]:

monet_names = tf.io.gfile.glob(r"../input/gan-getting-started/monet_tfrec/*.tfrec")
print(monet_names)
photo_names = tf.io.gfile.glob(r"../input/gan-getting-started/photo_tfrec/*.tfrec")

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) 
         for filename in filenames]
    return np.sum(n)

print(f"Monet file tfrecords: {len(monet_names)}")
print(f"Photo file tfrecords: {len(photo_names)}")
print(f"Monet images: {count_data_items(monet_names)}")
print(f"Photo images: {count_data_items(photo_names)}")

# Loading the data
- ***read_tfrecord()***:     takes a file and parse to get required variables.
- ***prepare_image()***:     takes an image in tensor datatype and returns after reshaping and normalizing it.
- ***get_dataset()***:       reads the TFRecords file and maps with the help of read_tfrecord function.
- _**data_augment()**_:      as the name suggests augment the image data of tensor type for better accuracy.
- _**get_gan_dataset()**_:   returns our main datasets which we will use for the training purposes

In [ ]:
def prepare_image(img, dim = 256):    
    img = tf.image.decode_jpeg(img, channels = 3)
    img = (tf.cast(img, tf.float32) / 255.0) - 1
    img = tf.reshape(img, [dim, dim, 3])
    return img

def read_tfrecord(example):
    tfrec_format = {
        'image' : tf.io.FixedLenFeature([], tf.string),
        'image_name' : tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.string)
    }   
    
    example = tf.io.parse_single_example(example, tfrec_format)
    image = prepare_image(example['image'])
    return image

In [ ]:
def load_dataset(filenames, labeled=True, ordered=False):
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return dataset

In [ ]:
monet_ds = load_dataset(monet_names, labeled=True).batch(1)
photo_ds = load_dataset(photo_names, labeled=True).batch(1)

# Dataset visualisation
- batch_visualisation(): shows the given number of images in given path

In [ ]:
base_path = '../input/gan-getting-started/'
monet_path = os.path.join(base_path, 'monet_jpg')
photo_path = os.path.join(base_path, 'photo_jpg')

In [ ]:
def batch_visualization(path, n_images, is_random=True, figsize=(16, 16)):
    plt.figure(figsize=figsize)
    
    w = int(n_images ** .5)
    h = math.ceil(n_images / w)
    
    all_names = os.listdir(path)
    
    image_names = all_names[:n_images]
    if is_random:
        image_names = random.sample(all_names, n_images)
    
    for ind, image_name in enumerate(image_names):
        img = cv2.imread(os.path.join(path, image_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
        plt.subplot(h, w, ind + 1)
        plt.imshow(img)
        plt.axis('off')
    
    plt.show()

In [ ]:
batch_visualization(monet_path,6)

In [ ]:
batch_visualization(photo_path,6)

## Individual images
- color_hist_visualization(): Gives the RGB bar graph of photo
- channels_visualization(): Gives the RGB channel graph of photo
- grayscale_visualization(): Converts the RGB channel to grayscale
- color_graph(): Gives the RGB histogram graph

In [ ]:
rand_monet = r"../input/gan-getting-started/monet_jpg/0260d15306.jpg"
rand_photo = r"../input/gan-getting-started/photo_jpg/000ded5c41.jpg"

In [ ]:
def color_hist_visualization(image_path, figsize=(16, 4)):
    plt.figure(figsize=figsize)
    layers = ['red', 'green', 'blue']
    
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    plt.subplot(1, 4, 1)
    plt.imshow(img)
    plt.axis('off')
    
    for i in range(len(layers)):
        plt.subplot(1, 4, i + 2)
        plt.hist(
            img[:, :, i].reshape(-1),
            bins=25,
            alpha=0.5,
            color=layers[i],
            density=True
        )
        plt.title(layers[i])
        plt.xlim(0, 255)
        plt.xticks([])
        plt.yticks([])
    plt.show()

In [ ]:
print("Monet: ")
color_hist_visualization(rand_monet)
print("\nPhoto: ")
color_hist_visualization(rand_photo)

In [ ]:
def channels_visualization(image_path, figsize=(16, 4)):
    plt.figure(figsize=figsize)
    
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    plt.subplot(1, 4, 1)
    plt.imshow(img)
    plt.axis('off')
    
    for i in range(3):
        plt.subplot(1, 4, i + 2)
        tmp_img = np.full_like(img, 0)
        tmp_img[:, :, i] = img[:, :, i]
        plt.imshow(tmp_img)
        plt.xlim(0, 255)
        plt.xticks([])
        plt.yticks([])
    plt.show()

In [ ]:
print("Monet: ")
channels_visualization(rand_monet)
print("\nPhoto: ")
channels_visualization(rand_photo)

In [ ]:
def grayscale_visualization(image_path, figsize=(16, 4)):
    plt.figure(figsize=figsize)
    
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    tmp_img = np.full_like(img, 0)
    for i in range(3):
        tmp_img[:, :, i] = img.mean(axis=-1)
    plt.imshow(tmp_img)
    plt.axis('off')
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    plt.subplot(1,3,3)
    plt.imshow(img)
    plt.axis("off")
    
    plt.show()

In [ ]:
print("Monet: ")
grayscale_visualization(rand_monet)
print("\nPhoto: ")
grayscale_visualization(rand_photo)

In [ ]:
def color_graph(image_path, figsize=(16, 4)):
    plt.figure(figsize=figsize)
    
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.axis('off')
    
    chans = cv2.split(img)
    colors = ("b", "g", "r")
    plt.subplot(1, 2, 2)
    plt.title("'Flattened' Color Histogram")
    plt.xlabel("Bins")
    plt.ylabel("# of Pixels")
    features = []
    # loop over the image channels
    for (chan, color) in zip(chans, colors):
        # create a histogram for the current channel and
        # concatenate the resulting histograms for each
        # channel
        hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
        features.extend(hist)
        # plot the histogram
        plt.plot(hist, color = color)
        plt.xlim([0, 256])
    
    plt.show()

In [ ]:
print("Monet: ")
color_graph(rand_monet)
print("\nPhoto: ")
color_graph(rand_photo)

# Training a CycleGAN
In the CycleGAN’s case, the architecture is complex, and as a result, we need a structure that allows us to keep accessing the original attributes and methods that we have defined. As a result, we will write out the CycleGAN as a Python class of its own with methods to build the Generator and Discriminator, and run the training.

For the training to execute we will need a seperate Generator() and discriminator() function which we will feed to CycleGAN as methods which in turn needs the upsample() and downsample() of image.

- For downsampling we are using the Conv2D() as primary layer and LeakyReLU() as activation
- For upsampling we are using the Conv2DTranspose() as primary layer and Dropout() at 0.3, ReLU() as secondary layers

For all of these I have made a utility script as this notebook kernel size was exceeding

In [ ]:
from shutil import copyfile
copyfile(src = "../usr/lib/monet_using_gan_utility/monet_using_gan_utility.py", dst = "../working/monet.py")
from monet import *

From here we are declaring generator and discriminator spaces of GANs which will work the same as encoder and decoder of autoencoders. To know more about autoencoders, [click here](https://www.kaggle.com/discussion/240924)

In [ ]:
generator_g = Generator()
tf.keras.utils.plot_model(generator_g, dpi=48)

In [ ]:
discriminator_g = Discriminator()
tf.keras.utils.plot_model(discriminator_g, dpi=48)

In [ ]:
with strategy.scope():
    monet_generator = Generator() # transforms photos to Monet-esque paintings
    photo_generator = Generator() # transforms Monet paintings to be more like photos

    monet_discriminator = Discriminator() # differentiates real Monet paintings and generated Monet paintings
    photo_discriminator = Discriminator() # differentiates real photos and generated photos

# Building the cycleGAN model
Image-to-image translation frameworks are frequently difficult to train because of the need for perfect pairs; the CycleGAN solves this by making this an unpaired domain translation.

The CycleGAN has three losses:
- Cycle-consistent, which measures the difference between the original image and an image translated into a different domain and back again
- Adversarial, which ensures realistic images
- Identity, which preserves the color space of the image

Practical applications of the CycleGAN include self-driving car training and exten- sions that allow us to create different styles of images during the translation process.

In [ ]:
class CycleGan(keras.Model):
    def __init__(
        self,
        monet_generator,
        photo_generator,
        monet_discriminator,
        photo_discriminator,
        lambda_cycle=10):
        
        super(CycleGan, self).__init__()
        self.m_gen = monet_generator
        self.p_gen = photo_generator
        self.m_disc = monet_discriminator
        self.p_disc = photo_discriminator
        self.lambda_cycle = lambda_cycle
        
    def compile(
        self,
        m_gen_optimizer,
        p_gen_optimizer,
        m_disc_optimizer,
        p_disc_optimizer,
        gen_loss_fn,
        disc_loss_fn,
        cycle_loss_fn,
        identity_loss_fn):
        
        super(CycleGan, self).compile()
        self.m_gen_optimizer = m_gen_optimizer
        self.p_gen_optimizer = p_gen_optimizer
        self.m_disc_optimizer = m_disc_optimizer
        self.p_disc_optimizer = p_disc_optimizer
        self.gen_loss_fn = gen_loss_fn
        self.disc_loss_fn = disc_loss_fn
        self.cycle_loss_fn = cycle_loss_fn
        self.identity_loss_fn = identity_loss_fn
        
    def train_step(self, batch_data):
        real_monet, real_photo = batch_data
        
        with tf.GradientTape(persistent=True) as tape:
            # photo to monet back to photo
            fake_monet = self.m_gen(real_photo, training=True)
            cycled_photo = self.p_gen(fake_monet, training=True)

            # monet to photo back to monet
            fake_photo = self.p_gen(real_monet, training=True)
            cycled_monet = self.m_gen(fake_photo, training=True)

            # generating itself
            same_monet = self.m_gen(real_monet, training=True)
            same_photo = self.p_gen(real_photo, training=True)

            # discriminator used to check, inputing real images
            disc_real_monet = self.m_disc(real_monet, training=True)
            disc_real_photo = self.p_disc(real_photo, training=True)

            # discriminator used to check, inputing fake images
            disc_fake_monet = self.m_disc(fake_monet, training=True)
            disc_fake_photo = self.p_disc(fake_photo, training=True)

            # evaluates generator loss
            monet_gen_loss = self.gen_loss_fn(disc_fake_monet)
            photo_gen_loss = self.gen_loss_fn(disc_fake_photo)

            # evaluates total cycle consistency loss
            total_cycle_loss = self.cycle_loss_fn(real_monet, cycled_monet, self.lambda_cycle) + self.cycle_loss_fn(real_photo, cycled_photo, self.lambda_cycle)

            # evaluates total generator loss
            total_monet_gen_loss = monet_gen_loss + total_cycle_loss + self.identity_loss_fn(real_monet, same_monet, self.lambda_cycle)
            total_photo_gen_loss = photo_gen_loss + total_cycle_loss + self.identity_loss_fn(real_photo, same_photo, self.lambda_cycle)

            # evaluates discriminator loss
            monet_disc_loss = self.disc_loss_fn(disc_real_monet, disc_fake_monet)
            photo_disc_loss = self.disc_loss_fn(disc_real_photo, disc_fake_photo)

        # Calculate the gradients for generator and discriminator
        monet_generator_gradients = tape.gradient(total_monet_gen_loss,
                                                  self.m_gen.trainable_variables)
        photo_generator_gradients = tape.gradient(total_photo_gen_loss,
                                                  self.p_gen.trainable_variables)

        monet_discriminator_gradients = tape.gradient(monet_disc_loss,
                                                      self.m_disc.trainable_variables)
        photo_discriminator_gradients = tape.gradient(photo_disc_loss,
                                                      self.p_disc.trainable_variables)

        # Apply the gradients to the optimizer
        self.m_gen_optimizer.apply_gradients(zip(monet_generator_gradients,
                                                 self.m_gen.trainable_variables))

        self.p_gen_optimizer.apply_gradients(zip(photo_generator_gradients,
                                                 self.p_gen.trainable_variables))

        self.m_disc_optimizer.apply_gradients(zip(monet_discriminator_gradients,
                                                  self.m_disc.trainable_variables))

        self.p_disc_optimizer.apply_gradients(zip(photo_discriminator_gradients,
                                                  self.p_disc.trainable_variables))
        
        return {
            "monet_gen_loss": total_monet_gen_loss,
            "photo_gen_loss": total_photo_gen_loss,
            "monet_disc_loss": monet_disc_loss,
            "photo_disc_loss": photo_disc_loss
        }

In [ ]:
with strategy.scope():
    def discriminator_loss(real, generated):
        real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(tf.ones_like(real), real)
        generated_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(tf.zeros_like(generated), generated)
        total_disc_loss = real_loss + generated_loss
        return total_disc_loss * 0.5

In [ ]:
with strategy.scope():
    def generator_loss(generated):
        return tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(tf.ones_like(generated), generated)
    
with strategy.scope():
    def calc_cycle_loss(real_image, cycled_image, LAMBDA):
        loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
        return LAMBDA * loss1

with strategy.scope():
    def identity_loss(real_image, same_image, LAMBDA):
        loss = tf.reduce_mean(tf.abs(real_image - same_image))
        return LAMBDA * 0.5 * loss

Training

In [ ]:
with strategy.scope():
    monet_generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    photo_generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

    monet_discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    photo_discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
with strategy.scope():
    cycle_gan_model = CycleGan(
        monet_generator, photo_generator, monet_discriminator, photo_discriminator
    )

    cycle_gan_model.compile(
        m_gen_optimizer = monet_generator_optimizer,
        p_gen_optimizer = photo_generator_optimizer,
        m_disc_optimizer = monet_discriminator_optimizer,
        p_disc_optimizer = photo_discriminator_optimizer,
        gen_loss_fn = generator_loss,
        disc_loss_fn = discriminator_loss,
        cycle_loss_fn = calc_cycle_loss,
        identity_loss_fn = identity_loss
    )

In [ ]:
from IPython.display import clear_output

In [ ]:
try:
    cycle_gan_model.fit(
        tf.data.Dataset.zip((monet_ds, photo_ds)),
        epochs=30)
except:
    clear_output()

In [ ]:
print("Above fitting gives an error:\nInvalidArgumentError: Unable to parse tensor proto [Op:DatasetCardinality]")

All the solutions are cordially invited!